In [1]:
!pip install sqlalchemy
!pip install termcolor

In [12]:
from bs4 import BeautifulSoup 
import urllib.request
import ssl
from dotenv import load_dotenv
import openai
import os
import requests
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import pandas as pd
import json
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re
from sqlalchemy import inspect
import sqlalchemy
from sqlalchemy import text 
import dspy
from termcolor import colored

In [13]:
from dsp.modules.cache_utils import cache_turn_on

cache_turn_on

True

In [4]:
ssl._create_default_https_context = ssl._create_stdlib_context
html_link = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datacurrent.html"

with urllib.request.urlopen(html_link) as url:
    s = url.read()
    # I'm guessing this would output the html source code ?
    soup = BeautifulSoup(s,"lxml")

html_table = soup.find_all("table")
req_table = html_table[1]
hrefs_list = req_table.find_all('a')

In [5]:
req_href = {"US":[],"Europe":[],"Japan":[],"AUS_NZ_CANADA":[],"Emerging":[],"China":[],"India":[],"Global":[]}

for i in hrefs_list:
    name = i.get_text().strip()
    try:
        href_attr = i['href']
        # Only get the excel files
        if href_attr.endswith('.xls'):
            if "US" in name:
                req_href["US"].append(href_attr)
            elif "Europe" in name:
                req_href["Europe"].append(href_attr)
            elif "Japan" in name:
                req_href["Japan"].append(href_attr)
            elif "Aus" in name:
                req_href['AUS_NZ_CANADA'].append(href_attr)
            elif "Emerging" in name:
                req_href['Emerging'].append(href_attr)
            elif "China" in name:
                req_href['China'].append(href_attr)
            elif "India" in name:
                req_href['India'].append(href_attr)
            elif "Global" in name: 
                req_href['Global'].append(href_attr)
    except:
        pass

In [ ]:
ssl._create_default_https_context = ssl._create_stdlib_context

os.makedirs("DATA",exist_ok=True)
for country,excel_files in req_href.items():
    country_path = os.path.join("DATA",country) 
    os.makedirs(country_path,exist_ok=True)
    for file in excel_files:
        file_name = file.split("/")[-1].split(".")[0]
        full_file_name = os.path.join(country_path,f"{file_name}.xls")
        resp = requests.get(file,verify=False)
        output = open(full_file_name, 'wb')
        output.write(resp.content)
        output.close()

In [ ]:
for country in os.listdir("DATA"):
    dir_len = len(os.listdir(os.path.join("DATA",country)))
    country_len = len(req_href[country])
    print(f'FOR {country} WE HAVE DIRECTORY LEN = {dir_len} and ACTUAL LEN = {country_len}')

In [ ]:
sample_excel = pd.ExcelFile("DATA/US/capex.xls")
sn = 'Variables & FAQ'
sample_excel.parse(sn).head(10)

In [5]:
sample_excel.parse(sample_excel.sheet_names[1]).head(10)

,Date updated:,2024-01-05 00:00:00,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Created by:,"Aswath Damodaran, adamodar@stern.nyu.edu",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,What is this data?,"Capital Expenditures, Acquisitions and R&D and...",NaN,NaN,NaN,US companies,NaN,NaN,NaN,NaN
2,Home Page:,http://www.damodaran.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Data website:,https://pages.stern.nyu.edu/~adamodar/New_Home...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Companies in each industry:,https://pages.stern.nyu.edu/~adamodar/pc/datas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Variable definitions:,https://pages.stern.nyu.edu/~adamodar/New_Home...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Industry Name,Number of Firms,Capital Expenditures (US $ millions),Depreciation & Amort ((US $ millions),Cap Ex/Deprecn,Acquisitions (US $ millions),Net R&D (US $ millions),Net Cap Ex/Sales,Net Cap Ex/ EBIT (1-t),Sales/ Invested Capital (LTM)
7,Advertising,57,775.729,1887.338,0.411018,322.559,75.08,-0.016886,-0.21812,3.283403
8,Aerospace/Defense,70,10982.128,13311.598,0.825004,10344.75,830.4634,0.022829,0.318077,1.98434
9,Air Transport,25,25559.725,10609.948,2.409034,368.21,73.5486,0.067203,1.355173,1.77732


In [7]:
import pandas as pd
pd.set_option('display.max_rows', 50)

def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)

dir = "DATA"
processed_dir = "Processed Data"
all_infos_dict = []
os.makedirs(processed_dir,exist_ok=True)
for country in os.listdir(dir):
    print(country)
    file_name = os.path.join(dir,country)
    os.makedirs(os.path.join(processed_dir,country),exist_ok=True)
    os.makedirs(file_name,exist_ok=True)
    for excel_file in tqdm(os.listdir(file_name)):
        full_file_name = os.path.join(file_name,excel_file)
        xls = pd.ExcelFile(full_file_name)
        sns = xls.sheet_names
        info_dict = {}
        for sheet_name in sns:
            if "Var" in sheet_name or "var" in sheet_name:
                info_df = xls.parse(sheet_name)
                info_df.dropna(how="all",inplace=True)
                info_dict = {}
                for cols in info_df.columns:
                    if "End" not in cols and 'Unnamed' not in cols:
                        info_dict['Summary'] = cols
                info_dict['Vars'] = info_df.values[1:].tolist()
                all_infos_dict.append(info_dict)
            elif "Industry" in sheet_name or "industry" in sheet_name:
                data_df = xls.parse(sheet_name)
        try:
            data_df.dropna(axis=1,thresh=5,inplace=True)
            data_df.dropna(inplace=True)
            new_header = data_df.iloc[0] #grab the first row for the header
        except:
            print(full_file_name)
            print(data_df)

        data_df = data_df[1:] #take the data less the header row
        data_df.reset_index(inplace=True,drop=True)
        new_header = [sanitize_column_name(str(col)) for col in new_header]
        data_df.columns = new_header #set the header row as the df header
        save_name = full_file_name.split(".")[0].split("\\")
        save_name = save_name[-1]
        save_file_path = os.path.join(os.path.join(processed_dir,country),save_name)
        data_df.to_csv(save_file_path+".csv",index=False)
        with open(save_file_path+".json", "w") as outfile: 
            json.dump(info_dict, outfile)

AUS_NZ_CANADA


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 24.01it/s]


China


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 22.20it/s]


Emerging


...

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 24.74it/s]


Japan


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 24.37it/s]


US


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 22.85it/s]


In [8]:
all_infos_dict[0]
# save_name

{'Summary': 'To measure how much companies are reinvesting back into their long term assets, as a prelude to forecasting expected growth.',
 'Vars': [['Capital Expenditures',
   'Sum of the capital expenditures reported on statement of cash flows',
   'Gross investment in long term assets, at least as defined by accountants',
   '$ millions'],
  ['Depreciation',
   'Sum of the depreciation reported on statement of cash flows',
   'Loss in value of assets, from use, as measured by accountants.',
   '$ millions'],
  ['Net Cap Ex',
   'Sum of capital expenditures on statement of cash flows minus Sum of depreciation as reported in statement of cash flows',
   'Net investment in long term assets, at least as defined by accountants',
   '$ millions'],
  ['Net R&D',
   'Sum of R&D reported as expense in most recent year minus amortization of R&D, assuming capitalization',
   'Net investment in long term assets, expanded to incorporate assets created by R&D',
   '$ millions'],
  ['Acquisitions

In [8]:
df = pd.read_csv("Processed Data/US/capex.csv")
df.head()

,Industry_Name,Number_of_Firms,Capital_Expenditures_US_millions_,Depreciation_Amort_US_millions_,Cap_Ex_Deprecn,Acquisitions_US_millions_,Net_R_D_US_millions_,Net_Cap_Ex_Sales,Net_Cap_Ex_EBIT_1_t_,Sales_Invested_Capital_LTM_
0,Advertising,57,775.729,1887.338,0.411018,322.559,75.0800,-0.016886,-0.218120,3.283403
1,Aerospace/Defense,70,10982.128,13311.598,0.825004,10344.750,830.4634,0.022829,0.318077,1.984340
2,Air Transport,25,25559.725,10609.948,2.409034,368.210,73.5486,0.067203,1.355173,1.777320
3,Apparel,38,1730.981,1386.480,1.248472,38.739,0.9474,0.005365,0.072557,1.773076
4,Auto & Truck,34,29899.486,18677.668,1.600815,193.220,983.0696,0.026577,0.675918,1.048732


In [9]:
import os

TOKEN="TOKEN"
ENDPOINT = 'https://models.inference.ai.azure.com'

os.environ['GITHUB_TOKEN']= TOKEN
os.environ["OPENAI_API_KEY"] = TOKEN

os.environ['OPENAI_API_BASE'] = ENDPOINT

In [10]:
model = dspy.LM('openai/gpt-4o-mini', max_tokens=250)
dspy.settings.configure(lm=model)

class SQLTableMetadata(dspy.Signature):
    """Give a suitable table name and description about the given table"""
    pandas_dataframe_str = dspy.InputField(desc="First 10 rows of a pandas dataframe delimited by newline character")
    table_name = dspy.OutputField(desc="suitable table name")
    table_summary = dspy.OutputField(desc="a summary about the table")

class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(SQLTableMetadata)
    
    def forward(self, pandas_dataframe_str):
        return self.prog(pandas_dataframe_str=pandas_dataframe_str)

cot = CoT()

In [ ]:
processed_dir = "Processed Data"
dfs_str = []
for country in os.listdir(processed_dir):
    country_folder = os.path.join(processed_dir,country)
    # print(f"{country}")
    for files in tqdm(os.listdir(country_folder),desc=f"Building the summary and name for {country}"):
        if files.endswith(".csv"):
            file_name = files.split(".")[0]
            csv_file_path = os.path.join(country_folder,files)
            df = pd.read_csv(csv_file_path,index_col=False)
            json_file_path = os.path.join(country_folder,f"{file_name}.json")
            with open(json_file_path,'r') as f:
                data = json.loads(f.read())
            if 'table_name' in data and 'table_summary' in data:
                # if data['table_name'] == "" or data['table_summary'] == "":
                if data['table_summary'] == "":
                    pass
                else:
                    continue
            dfs_str.append(df.head(10).to_csv())
            table_preds = cot(pandas_dataframe_str = df.head(10).to_csv())
            data['table_name'] = table_preds.table_name
            data['table_summary'] = table_preds.table_summary
            with open(json_file_path,'w') as f:
                json.dump(data, f)

In [ ]:
country_folder = "Processed_Data/AUS_NZ_CANADA"
for files in tqdm(os.listdir(country_folder),desc=f"Building the summary and name for {country}"):
    if files.endswith(".csv"):
        file_name = files.split(".")[0]
        csv_file_path = os.path.join(country_folder,files)
        df = pd.read_csv(csv_file_path,index_col=False)
        json_file_path = os.path.join(country_folder,f"{file_name}.json")
        with open(json_file_path,'r') as f:
            data = json.loads(f.read())
        if 'table_name' in data and 'table_summary' in data:
            # if data['table_name'] == "" or data['table_summary'] == "":
            if data['table_summary'] == "":
                pass
            else:
                continue
        dfs_str.append(df.head(10).to_csv())
        table_preds = cot(pandas_dataframe_str = df.head(10).to_csv())
        data['table_name'] = table_preds.table_name
        data['table_summary'] = table_preds.table_summary
        with open(json_file_path,'w') as f:
            json.dump(data, f)

In [17]:
# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()

In [18]:
processed_dir  = "Processed Data"
def sqlalchemy_engine(region:str):
    """Create a SQLAlchemy engine for the given region"""
    assert region in os.listdir(processed_dir), f"{region} is not a valid region from {os.listdir(processed_dir)}"
    # Create a SQLAlchemy database for each region
    engine = create_engine(f"sqlite:///{region}.db")
    metadata_obj = MetaData()
    region_path = os.path.join(processed_dir,region)
    dfs = []
    for dataframes_path in os.listdir(region_path):
        if dataframes_path.endswith(".csv"):
            df = pd.read_csv(os.path.join(region_path,dataframes_path),index_col=False)
            dfs.append((dataframes_path,df))
    pbar = tqdm(total=len(dfs),desc=f"Creating tables for {region}")
    for _, df_table_name in enumerate(dfs):
        table_name = df_table_name[0]
        table_name = table_name.split(".")[0]
        df = df_table_name[1]
        # print(f"Creating table: {table_name}")
        create_table_from_dataframe(df,table_name, engine, metadata_obj)
        # print(f"Done creating table for: {table_name}")
        pbar.update(1)
    return engine

In [21]:
# us_engine = sqlalchemy_engine("US")
india_engine = sqlalchemy_engine("India")
# china_engine = sqlalchemy_engine("China")
# europe_engine = sqlalchemy_engine("Europe")
# global_engine = sqlalchemy_engine("Global")
# aus_nz_canada_engine = sqlalchemy_engine("AUS_NZ_CANADA")
# japan_engine = sqlalchemy_engine("Japan")
# emerging_engine = sqlalchemy_engine("Emerging")

Creating tables for India: 100%|███████████████████████████████████████████████████████| 29/29 [00:01<00:00, 19.21it/s]


In [58]:
def get_table_infos(sql_engine:sqlalchemy.engine.base.Engine,region:str):
    """Get all the tables info in the database based on the given region"""
    inspector = inspect(sql_engine)
    table_names = inspector.get_table_names()
    table_infos_dict = {tb: [] for tb in table_names}
    for tb in table_names:
        column_dict = inspector.get_columns(tb)
        schema_str = ""
        primary_keys = []
        for col in column_dict:
            schema_str += f"{col['name']} ({col['type']}), "
            if col["primary_key"] not in primary_keys:
                primary_keys.append(col["name"])
        
        with open(os.path.join(processed_dir,region,f"{tb}.json")) as f:
            table_info = json.loads(f.read())
        table_info = list(table_info.items())
        # print(table_info)
        table_infos_dict[tb] = [
            {
                "table_info": f"Table {tb} has columns: {schema_str[:-2]}",
                "table_summary": f'{table_info[0][-1]}. {table_info[-1][-1]}. ',
            }
        ]
    return table_infos_dict

In [ ]:
i = inspect(india_engine)
i.reflect_table?

In [59]:
# us_tb_dict = get_table_infos(us_engine,"US")
india_tb_dict = get_table_infos(india_engine,"India")
# china_tb_dict = get_table_infos(china_engine,"China")
# europe_tb_dict = get_table_infos(europe_engine,"Europe")
# global_tb_dict = get_table_infos(global_engine,"Global")
# aus_nz_canada_tb_dict = get_table_infos(aus_nz_canada_engine,"AUS_NZ_CANADA")
# japan_tb_dict = get_table_infos(japan_engine,"Japan")
# emerging_tb_dict = get_table_infos(emerging_engine,"Emerging")

{'name': 'Net_Income_millions_', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'D_D_E_', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'Stock_based_Compensation_as_of_Revenue', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': '2023_0', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'CAGR_in_R_D_Last_5_years', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'Average_Beta_2019_23', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'Sales_Invested_Capital_LTM_', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'Capital_Spending_Total_Assets', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'Lease_Debt_Accounting_', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'Cash_Firm_Value', 'type': INTEGER

In [ ]:
india_tb_dict